# XZZX threshold vs bias

The XZZX surface code introduced in [Bonilla Atiades, Tuckett, Bartlett, Flammia and Brown 2021](https://arxiv.org/abs/2009.07851) is an example of a Clifford-deformed variant of the 2D surface code with ultra-high thresholds for Pauli noise under high bias.
In this tutorial, we explore how to use PanQEC to compute the threshold error rates of the XZZX surface code and the CSS surface code under Pauli noise of different values of bias.
The goal is to produce a plot of the threshold physical error rate vs the Pauli channel $Z$ bias for both the CSS surface code and the XZZX surface code for their performance to be compared.

The CSS toric code is found in PanQEC as the class [panqec.codes.Toric2DCode](/codes.html#panqec.codes.surface_2d.Toric2DCode).

In [17]:
from panqec.codes import Toric2DCode

A one-qubit Pauli channel takes the form
$$\mathcal{E}(\rho)  = (1 - p) \rho + p (r_X X \rho X + r_Y Y \rho Y + r_z Z \rho Z)$$
where $p$ is the physical error rate and $r_X, r_Y, r_Z\in [0, 1]$ are parameters such that  $r_X + r_Y + r_Z=1$.
Physically, this means that when this channel is applied on a qubit,
with probably $p$ an error will occur,
which could be either Pauli $X,Y,Z$ with probability $r_X,r_Y,r_Z$.
This error model is included in PanQEC as [panqec.error_models.PauliErrorModel](/error_models.html#panqec.error_models.PauliErrorModel).

In [15]:
from panqec.error_models import PauliErrorModel

The _bias_ of a channel is the ratio
$$\eta_Z = \frac{r_Z}{r_X + r_Y}$$
For a depolarizing channel $\eta_Z=0.5$ and for a dephasing channel $\eta_Z=\infty$.
For this tutorial,
we will assume that $r_X=r_Y$ so then
$$\eta_Z = \frac{r_Z}{2r_X}$$

When using the matching decoder, we have already shown how to use PanQEC to compute the threshold error rate for the CSS toric code under biased noise in the previous tutorial
[Computing the threshold of the surface code](/tutorials/Computing%20threshold.html).

However, this time we will use the BP-OSD decoder ([Pantaleev and Kalachev 2021](http://arxiv.org/abs/1904.02703) and [Roffe, White, Burton and Campbell 2020](https://arxiv.org/abs/2005.07016)),
in particular the [open-source Roffe implementation](https://github.com/quantumgizmos/bp_osd),
which is included with PanQEC as [panqec.decoders.BeliefPropagationOSDDecoder](/decoders.html#panqec.decoders.BeliefPropagationOSDDecoder)

In [20]:
from panqec.decoders import BeliefPropagationOSDDecoder

Since we already have a working decoder and CSS code,
it is convenient to not to have to rewrite the decoder and code when studying the Clifford-deformed XZZX code,
which is merely related to the CSS surface code by a Hadamard on every qubit along a particular axis.

As such, to simplify the simulations and reuse the existing code and decoder,
instead of deforming the code,
we can deform the noise model,
which will give equivalent statistics.
Luckily in the PanQEC,
deformations of codes can be specified in the class definition of the code itself.
We can view what deformations are available for the code.

In [21]:
Toric2DCode.deformation_names

['XZZX', 'XY']

We can see that we have two deformations implemented,
namely the XZZX deformation and the XY deformation ([Tuckett, Bartlett and Flammia 2017](https://arxiv.org/abs/1708.08474)).

We can now specify this in the input data in the `error_model` attribute.

In [23]:
import numpy as np
input_data = {
    'ranges': {
        'label': 'Toric 2D Experiment',  # Can be any name you want
        'code': {
            'name': 'Toric2DCode',  # Class name of the code
            'parameters': [{'L_x': 6}, {'L_x': 12}, {'L_x': 24}]
        },
        'error_model': {
            'name': 'PauliErrorModel',  # Class name of the error model
            'parameters': [
                {'r_x': 1/3, 'r_y': 1/3, 'r_z': 1/3, 'deformation_name': 'XZZX'}  # Ratios of X, Y and Z errors
            ],
        },
        'decoder': {
            'name': 'MatchingDecoder',  # Class name of the decoder
            'parameters': [{}]
        },
        'error_rate': np.linspace(0.1, 0.2, 10).tolist()  # List of physical error rates
    }
}

However, we want to do many values of bias.

In [11]:
from panqec.utils import get_direction_from_bias_ratio
get_direction_from_bias_ratio('Z', 10)

In [27]:
[get_direction_from_bias_ratio('Z', eta) for eta in [0.5, 10, 100, np.inf]]

[{'r_x': 0.33333333333333337,
  'r_y': 0.33333333333333337,
  'r_z': 0.3333333333333333},
 {'r_x': 0.04545454545454547,
  'r_y': 0.04545454545454547,
  'r_z': 0.9090909090909091},
 {'r_x': 0.004950495049504955,
  'r_y': 0.004950495049504955,
  'r_z': 0.9900990099009901},
 {'r_x': 0.0, 'r_y': 0.0, 'r_z': 1.0}]

In [1]:
from tqdm import tqdm

In [3]:
from panqec.simulation import read_input_dict
from panqec.analysis import Analysis

In [7]:
plot_frequency = 20  # Frequency of plot update
save_frequency = 10  # Frequency of saving to file
n_trials = 500  # Target number of Monte Carlo runs

# We create a BatchSimulation by reading the input dictionary
batch_sim = read_input_dict(
    input_data,
    output_file='xzzx-2d-results.json',  # Where to store the simulation results
    update_frequency=plot_frequency,
    save_frequency=save_frequency
)

# Live update of the plot during the simulation
# (only works in Jupyter notebooks)
batch_sim.activate_live_update()

Start batch simulation instance


In [ ]:
batch_sim.run(n_trials, progress=tqdm)

In [9]:
analysis = Analysis("toric-2d-results.json")

In [ ]:
analysis.make_collapse_plots()